In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import json
import pandas as pd
import os
import re
import unicodedata

In [3]:
# Load the JSON data
def load_json_data(filepath: str) -> list:
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            report_data = json.load(f)

        if report_data:
            if isinstance(report_data, list):
                print(f"Successfully loaded JSON. Found {len(report_data)} records.")
            else:
                print(f"Loaded JSON, but it's not a list. Type: {type(report_data)}. Handling appropriately.")
                report_data = [report_data] if isinstance(report_data, dict) else []
        else:
            print("WARNING: JSON file was loaded but appears to be empty or None.")
            report_data = []
        return report_data

    except FileNotFoundError:
        print(f"ERROR: File not found at '{filepath}'. Please check the path.")
    except json.JSONDecodeError as e:
        print(f"ERROR: Could not decode JSON from file {filepath}. The file may be corrupted. Error: {e}")
    except Exception as e:
        print(f"ERROR: An unexpected error occurred during file loading: {e}")
    return []

In [4]:
# --- ACTIONABILITY & SPECIFICITY ---
# Load the master DataFrame for actionability and specificity
master_df = pd.DataFrame(load_json_data('dataset/preprocessed/master_df.json'))
actionability_df = pd.DataFrame(load_json_data('dataset/evaluations/actionability_results.json'))
specificity_df = pd.DataFrame(load_json_data('dataset/evaluations/specificity_results.json'))


# --- RUBRIC ALIGNMENT & FACTIONAL CONSISTENCY ---
# Load the master DataFrame for rubric alignment and factual consistency
master_second_df = pd.DataFrame(load_json_data('dataset/preprocessed/master_second_df.json'))
rubric_alignment_df = pd.DataFrame(load_json_data('dataset/evaluations/rubric_alignment_results.json'))
factual_consistency_df = pd.DataFrame(load_json_data('dataset/evaluations/factual_consistency_results.json'))

Successfully loaded JSON. Found 8634 records.
Successfully loaded JSON. Found 8634 records.
Successfully loaded JSON. Found 8634 records.
Successfully loaded JSON. Found 7023 records.
Successfully loaded JSON. Found 7023 records.
Successfully loaded JSON. Found 7023 records.


In [5]:
# --- Prepare the DataFrames for Merging ---
# Ensure both DataFrames have a 'paper_content' column for merging.
# Rename 'input' to 'paper_content' for clarity.
actionability_df.rename(columns={'actual_output': 'comment'}, inplace=True)
specificity_df.rename(columns={'actual_output': 'comment'}, inplace=True)
rubric_alignment_df.rename(columns={'actual_output': 'comment'}, inplace=True)
factual_consistency_df.rename(columns={'input': 'comment'}, inplace=True)


# Now, merge the two DataFrames by finding rows with the exact same 'comment' text.
# This is a foolproof way to align the rows correctly, regardless of their original order.
merged_actionability_df = pd.merge(actionability_df, master_df, on='comment', how='left')
merged_specificity_df = pd.merge(specificity_df, master_df, on='comment', how='left')
merged_rubric_alignment_df = pd.merge(rubric_alignment_df, master_second_df, on='comment', how='left')
merged_factual_consistency_df = pd.merge(factual_consistency_df, master_second_df, on='comment', how='left')



# --- Display the Final, Correctly Merged DataFrame ---
# Sort the merged DataFrames by 'assignment_id' and 'item' for better readability.
# This will ensure that the rows are in a consistent order.
merged_actionability_df.sort_values(['assignment_id','item'], inplace=True)
merged_actionability_df.reset_index(drop=True, inplace=True)
merged_actionability_df.rename(columns={'score_x': 'judge_score'}, inplace=True)
merged_actionability_df.rename(columns={'score_y': 'grader_score'}, inplace=True)


merged_specificity_df.sort_values(['assignment_id','item'], inplace=True)
merged_specificity_df.reset_index(drop=True, inplace=True)
merged_specificity_df.rename(columns={'score_x': 'judge_score'}, inplace=True)
merged_specificity_df.rename(columns={'score_y': 'grader_score'}, inplace=True)


merged_rubric_alignment_df.sort_values(['assignment_id','item'], inplace=True)
merged_rubric_alignment_df.reset_index(drop=True, inplace=True)
merged_rubric_alignment_df.rename(columns={'score_x': 'judge_score'}, inplace=True)
merged_rubric_alignment_df.rename(columns={'score_y': 'grader_score'}, inplace=True)


merged_factual_consistency_df.sort_values(['assignment_id','item'], inplace=True)
merged_factual_consistency_df.reset_index(drop=True, inplace=True)
merged_factual_consistency_df.rename(columns={'score_x': 'judge_score'}, inplace=True)
merged_factual_consistency_df.rename(columns={'score_y': 'grader_score'}, inplace=True)


# --- Final Cleanup of Columns ---
# Keep only the necessary columns for the final DataFrame.
final_columns_to_keep = [
    'assignment_id',
    'paper_content',
    'item',
    'type', 
    'grader_score',
    'comment',
    'evidence',
    'score_range',
    'rubric',
    'metric',
    'judge_score',
    'status',
    'reason'
    ]


# Filter the DataFrames to keep only the final columns.
# This ensures that we have a consistent structure across all merged DataFrames.
merged_actionability_df = merged_actionability_df[final_columns_to_keep]
merged_specificity_df = merged_specificity_df[final_columns_to_keep]
merged_rubric_alignment_df = merged_rubric_alignment_df[final_columns_to_keep]
merged_factual_consistency_df = merged_factual_consistency_df[final_columns_to_keep]


# --- Save the Final Merged DataFrames ---
merged_actionability_df.to_json('dataset/merged/merged_actionability.json', orient='records', indent=4)
merged_specificity_df.to_json('dataset/merged/merged_specificity.json', orient='records', indent=4)
merged_rubric_alignment_df.to_json('dataset/merged/merged_rubric_alignment.json', orient='records', indent=4)
merged_factual_consistency_df.to_json('dataset/merged/merged_factual_consistency.json', orient='records', indent=4)


merged_actionability_df.info()
merged_specificity_df.info()
merged_rubric_alignment_df.info()
merged_factual_consistency_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8634 entries, 0 to 8633
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   assignment_id  8634 non-null   int64  
 1   paper_content  8634 non-null   object 
 2   item           8634 non-null   object 
 3   type           8634 non-null   object 
 4   grader_score   8634 non-null   float64
 5   comment        8634 non-null   object 
 6   evidence       7023 non-null   object 
 7   score_range    7023 non-null   object 
 8   rubric         7023 non-null   object 
 9   metric         8634 non-null   object 
 10  judge_score    8634 non-null   float64
 11  status         8634 non-null   object 
 12  reason         8634 non-null   object 
dtypes: float64(2), int64(1), object(10)
memory usage: 877.0+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8634 entries, 0 to 8633
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------      